In [1]:
import os.path
import sys
sys.path.append('/Users/cmagri1/OneDrive - Johns Hopkins/git')
from torch import nn
import torch
from torchvision.models.alexnet import alexnet
from tqdm import tqdm

from EncodingModel_cm296 import utils as emutils
from EncodingModel_cm296 import feature_extractor as emfe

import pandas as pd
import numpy as np
import csv

from utilsCM import Subject
from utilsCM import cv_regression #Predicting fMRI reponses with features (Alexnet activations) 
#cv_regression computes cross-validated ridge regression. cross-validation groupings were pre-set based
#on fMRI design. 9-categories out, 9 folds. r is averaged over folds, weights are computed over all data


In [2]:
# specify variables
resolutionval = 227;
# print(model)

#sepcify directory where images (for which we have fMRI data) are
PathToImgs = 'data-object2vec/stimuli'

In [3]:
#Load word2sense
#already preprocessed in MATLAB so that Wrd2Sns and THINGs overlap --> we have IMAGES, LABELS and SENSES
pathtofile = '../code-00-preprocessdataset/'
Wrd2Sense = pd.read_csv(pathtofile + "ThingsWrd2Sns.txt", sep=",")
ImgInfo = pd.read_csv(pathtofile + "KeptTHINGSInfo.txt", sep=",")
WordInf= ImgInfo.Word.to_numpy() ##to find the words that are images in THINGS dataset
pathtoTHINGS = '/Users/cmagri1/OneDrive - Johns Hopkins/Project-Word2Sense/THINGSdataset/Main/images'
savepath = '../../../data-00/'
pretrainedModel = False

In [4]:
#Compute DNN features
def object_by_feature(PathToImgs,keyword,ilayer,resolutionval, savepath,WordInf=[],pretrained=True):
    if os.path.isfile(savepath+ keyword + '_' + ilayer +"_untrained"+ ".csv"):
        print('loading file with '+ keyword+ ' stimuli features for ' + ilayer)
        features = pd.read_csv(savepath+ keyword + "_" + ilayer + ".csv"  , sep=",", header=None, index_col=0)
#         features = pd.read_csv(datapath+ keyword + "_" + ilayer + ".csv"  , sep=",", header=None)

        condition_features = {}
        for index, row in features.iterrows():
            condition_features[index] = row 
    else:
        print('Computing '+ keyword+ ' stimuli features for ' + ilayer)
        
        #Specify the model
        model = emfe.AlexNet(ilayer,pretrained_val=pretrained); 
        
        if WordInf != np.array([]):
            condition_features = emutils.compute_features(model, PathToImgs,resolutionval,WordInf)
        else:
            condition_features = emutils.compute_features(model, PathToImgs,resolutionval)
        features = pd.DataFrame(condition_features).transpose() 
        
        if pretrained:
            features.to_csv(savepath + keyword+ "_" + ilayer+ ".csv", index = True, header=False)
            np.save(savepath + keyword+ "_" + ilayer,condition_features)
        else:
            features.to_csv(savepath + keyword+ "_" + ilayer +"_untrained"+ ".csv", index = True, header=False)
            np.save(savepath + keyword+ "_" + ilayer+"_untrained",condition_features)
#     layer_by_features[ilayer] = features; #create multistructure with all layers
    return condition_features

In [8]:
# layer = {'conv_1','conv_2','conv_3','conv_4','conv_5','fc_1','fc_2','fc_3'};
layer = {'conv_1','conv_5'};

ROI = {'PPA'}
Sub = [1]

In [ ]:
for ilayer in layer:    

     #Compute features of CNN
     #---------Object2Vect-----------------
    keyword = 'object2vec'
    object2vec_features = object_by_feature(PathToImgs,keyword,ilayer,resolutionval,savepath,pretrained=pretrainedModel)
#     #---------THINGS-----------------
    keyword = 'things'
    things_features = object_by_feature(pathtoTHINGS,keyword,ilayer,resolutionval,savepath,pretrained=pretrainedModel)
    things_features_df = pd.DataFrame(things_features).transpose()
    
    for iSub in Sub:
        for iROI in ROI:
            subject = Subject(iSub,[iROI])
            #predict fMRI using object2vec DNN activations
            weights, r = cv_regression(object2vec_features, subject, l2=0)
            
            #     #---Compute Predicted ROI response to THINGS dataset
            print("Computing ROI " + iROI + " prediction for " + ilayer)    
            ROIpred = {}
            ROIpred = np.matmul(things_features_df.iloc[:,:].to_numpy(),weights.transpose())
            
            if pretrainedModel:
                np.save(savepath + '/ROIpred_Sub' + str(iSub) + '_' + iROI + "_" + ilayer, ROIpred)
            else:
                np.save(savepath + '/ROIpred_Sub' + str(iSub) + '_' + iROI + "_" + ilayer + '_untrained', ROIpred)
print('things dataset activations in DNN '+ ilayer +' size: ' + str(things_features_df.shape))
print('weights size: ' + str(weights.shape))
print('ROI pred size: ' + str(ROIpred.shape))
print("Done!")